## Implementation of deep generative models for Recommender Systems

https://medium.com/snipfeed/how-to-implement-deep-generative-models-for-recommender-systems-29110be8971f

In [2]:
import tensorflow as tf

In [6]:
import tensorflow as tf

nbr_item = 100  # Define the value of nbr_item according to your specific case

generator = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(None, nbr_item)),
        tf.keras.layers.Dense(nbr_item, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(0.01))
    ]
)


In [7]:
discriminant = tf.keras.Sequential(
          [
              tf.keras.layers.InputLayer(input_shape=(None,nbr_item)),
              tf.keras.layers.Dense(1,kernel_regularizer = tf.keras.regularizers.l2(0.01))
          ]
        )

In [11]:
def generate(self,user,keep_greater_0 = False):                       
    g = self.generator(user)      

    if keep_greater_0:
        g = tf.multiply(g,tf.cast(user > 0,tf.float32))

    return g

In [12]:
def discriminate(self,user,use_sigmoid=True,add_noise=False):
    if add_noise:
        user = tf.clip_by_value(user + tf.random.truncated_normal(user.shape,mean=0.0,stddev=0.3),0,1)

    res = self.discriminant(user)

    if use_sigmoid:
        res = tf.nn.sigmoid(res)

    return res

In [13]:
def get_discriminant_variables(self):
    return self.discriminant.trainable_variables

In [14]:
def get_generator_variables(self):
    return self.generator.trainable_variables


In [15]:
def compute_loss_discriminant(self,user):    
    dis_real = self.discriminate(user)
    dis_fake = self.discriminate(self.generate(user,True))

    return - tf.reduce_mean(tf.log(1. - dis_fake + 10e-5) + tf.log(dis_real + 10e-5)) 

def compute_loss_generator(self,user): 
    gen_fake = self.discriminate(self.generate(user,True))
    
    return - tf.reduce_mean(tf.log(gen_fake+10e-5))

In [18]:
def compute_loss_discriminant(self,batch): 
        batch_fake = self.generate(batch,True)
        dis_real = self.discriminate(batch)
        dis_fake = self.discriminate(batch_fake)
             
        epsilon = tf.random_uniform(shape=(batch.shape[0], 1))
        
        batch_hat = epsilon * batch + (1. - epsilon) * batch_fake
        
        with tf.GradientTape() as g:
            g.watch(batch_hat)
            dis_batch_hat = self.discriminate(batch_hat)
        grad_dis_batch_hat = g.gradient(dis_batch_hat, batch_hat) 
                        
        grad_norm = tf.square(tf.norm(grad_dis_batch_hat,axis=-1) - 1)
        
        return tf.reduce_mean(dis_fake - dis_real + grad_norm * self.lambda_)
    
def compute_loss_generator(self,user):     
        return - tf.reduce_mean(self.discriminate(self.generate(user,True)))